<a href="https://colab.research.google.com/github/byoscar88888/SAA_entrega/blob/main/MIA/Ejercicio1.5puntos/Ejercicio_1_5puntos.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%%capture
%pip install transformers torch
%pip install langchain
%pip install langchain_hub
%pip install langchain -q
%pip install langchain_community
%pip install accelerate
%pip install chromadb



In [ ]:
# Importar la clase HuggingFacePipeline de la biblioteca langchain
from langchain import HuggingFacePipeline
# Creación de una plantilla de prompt con LangChain
from langchain import PromptTemplate
# Importar la clase PDFMinerLoader y CharacterTextSplitter de LangChain
from langchain.document_loaders import PDFMinerLoader
from langchain.text_splitter import CharacterTextSplitter
# Importar y crear las incrustaciones usando HuggingFaceEmbeddings
from langchain.embeddings import HuggingFaceEmbeddings
# Importar y crear la tienda de vectores Chroma
from langchain.vectorstores import Chroma
# Crear una cadena de recuperación conversacional utilizando LangChain
from langchain.chains import ConversationalRetrievalChain
# Utilizar herramientas de LangChain
from langchain.agents import load_tools, initialize_agent, AgentType


In [ ]:
from langchain import HuggingFacePipeline

llm = HuggingFacePipeline.from_model_id(model_id="stabilityai/stablelm-zephyr-3b",
                                        task="text-generation",
                                        model_kwargs={"temperature": 0.0, "max_length": 2048},
                                        pipeline_kwargs={"max_new_tokens": 2000})

c:\Users\User\AppData\Local\Programs\Python\Python312\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
c:\Users\User\AppData\Local\Programs\Python\Python312\Lib\site-packages\transformers\generation\configuration_utils.py:515: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.0` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`. This was detected when initializing the generation config instance, which means the corresponding file may hold incorrect parameterization and should be fixed.
  warnings.warn(
c:\Users\User\AppData\Local\Programs\Python\Python312\Lib\site-packages\transformers\generation\configura

In [ ]:
# Creación de una plantilla de prompt con LangChain
from langchain import PromptTemplate

# Definir una plantilla de prompt
template = """{prompt|>}{question}"""

prompt = PromptTemplate(template=template, input_variables=["question"])


In [ ]:
# Importar la clase PDFMinerLoader y CharacterTextSplitter de LangChain
from langchain.document_loaders import PDFMinerLoader
from langchain.text_splitter import CharacterTextSplitter

# Cargar y dividir el PDF en documentos
def load_and_split_documents(pdf_paths):
    documents = []
    for path in pdf_paths:
        loader = PDFMinerLoader(path)
        pdf = loader.load()
        text_splitter = CharacterTextSplitter(chunk_size=1024, chunk_overlap=64)
        documents.extend(text_splitter.split_documents(pdf))
    return documents

# Lista de rutas de PDFs en español
pdf_paths = ["C:/Users/User/Desktop/1110.2646v1.pdf","C:/Users/User/Desktop/Dialnet-ConceptoDePoliticaYVidaCotidiana-4953719.pdf"]  # Agrega aquí las rutas de tus PDFs

documents = load_and_split_documents(pdf_paths)
print(f"Número de documentos: {len(documents)}")

Número de documentos: 56


In [ ]:
# Importar y crear las incrustaciones usando HuggingFaceEmbeddings
from langchain.embeddings import HuggingFaceEmbeddings

# Crear el objeto de incrustaciones
embeddings = HuggingFaceEmbeddings()

# Incrustar la primera página del contenido de los documentos
query_result = embeddings.embed_query(documents[0].page_content)
print(query_result)

c:\Users\User\AppData\Local\Programs\Python\Python312\Lib\site-packages\huggingface_hub\file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


[-0.013656037859618664, -0.05266005918383598, -0.030925272032618523, 0.03265092149376869, -0.038084693253040314, 0.006587338633835316, 0.04278538376092911, 0.0569918155670166, -0.05068400874733925, -0.001986174378544092, -0.028200697153806686, -0.032039716839790344, 0.04509471356868744, 0.022067178040742874, 0.0229068361222744, -0.03340299800038338, -0.006155660375952721, -0.02216675691306591, 0.03439604118466377, -0.01125062070786953, -0.010984797030687332, -0.026215696707367897, -0.04136999323964119, -0.0241792481392622, -0.030391350388526917, -0.05217675492167473, -0.013243235647678375, -0.006290933582931757, -0.018663030117750168, -0.018596455454826355, -0.008110018447041512, 0.06412006169557571, -0.02269265055656433, 0.00877982098609209, 2.4359662802453386e-06, -0.04937320202589035, 0.006216530222445726, 0.006105224601924419, -0.009189234115183353, 0.014807489700615406, 0.03872988373041153, -0.02202606573700905, -0.019022047519683838, -0.04014749079942703, -0.056781403720378876, 0

In [ ]:
# Importar y crear la tienda de vectores Chroma
from langchain.vectorstores import Chroma

# Crear la tienda de vectores utilizando los documentos e incrustaciones
vectorstore = Chroma.from_documents(documents, embeddings)

# Crear una cadena de recuperación conversacional utilizando LangChain
from langchain.chains import ConversationalRetrievalChain

qa = ConversationalRetrievalChain.from_llm(
    llm,
    vectorstore.as_retriever(),
    return_source_documents=True
)

In [ ]:
# Ejemplo de uso de la cadena de recuperación conversacional
chat_history = []
query = "¿Definición de política?"
result = qa({"question": query, "chat_history": chat_history})
print(result["answer"])

# Mostrar el contenido de la página del primer documento fuente
print(result['source_documents'][0].page_content)

c:\Users\User\AppData\Local\Programs\Python\Python312\Lib\site-packages\langchain_core\_api\deprecation.py:119: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 0.3.0. Use invoke instead.
  warn_deprecated(


Use the following pieces of context to answer the question at the end. If you don't know the answer, just say that you don't know, don't try to make up an answer.

academia españolahttp://diccionario.terra.com.pe/cgi-bin/b.pl). 

Política  en  términos  generales  hace  alusión  al  grupo  de  actividades 

calificadas  como  políticas  es  decir  acciones  humanas  que  se  refieren  a 

las cosas del  Estado, de ahí, que también se le señale como doctrina o 

ciencia del Estado. 

Si  se  entiende  en  el  primer  sentido,  su  objeto  de  estudio  es  el  Estado, 

entendido como la institución que rige a los individuos que conforman un 

grupo  social  en  un  territorio  especifico,  sobre  los  cuales  le  corresponde 

legislar,  extraer  y  distribuir  los  recursos.  A  partir  de  este  significado 

surge,  por  un  lado  el  poder  estatal  que  defiende  y  mantiene,  conquista 

una ideología propia como se puede apreciar en las actuales contiendas 

políticas. 

Al  ente

In [ ]:
# Ejemplo de uso de la cadena de recuperación conversacional
chat_history = []
query = "¿Qué es un artículo científico?"
result = qa({"question": query, "chat_history": chat_history})
print(result["answer"])

# Mostrar el contenido de la página del primer documento fuente
print(result['source_documents'][0].page_content)

Use the following pieces of context to answer the question at the end. If you don't know the answer, just say that you don't know, don't try to make up an answer.

”

1

Un artículo científico es un trabajo de investigación (también llamado  paper) para ser 
publicado en una revista especializada (apegado a unas normas y arbitrado), en el cual 
se pretende comunicar un resultado original producto de una investigación y dirigido a 
terceras personas para transferir el conocimiento generado [1].

En   cuanto   a   un   artículo   de   revisión   (review),   también   tiene   la   modalidad   de   ser 
publicado  para  revistas  especializadas  (journals)  y debe  ser  evaluado  (arbitrado).  La 
diferencia con el artículo científico radica que en el  review  se dan a conocer las bases 
teóricas del tema deseado (el estado del arte) [1]. Algunas revistas exigen en los reviews 
contrastar   y   añadir   opiniones   con   criterios   propios,   agregando   valor   a   la   simple 
búsqueda

In [ ]:
# Utilizar herramientas de LangChain
from langchain.agents import load_tools, initialize_agent, AgentType

# Cargar herramientas adicionales
tools = load_tools(["arxiv"])

# Inicializar el agente
agent_chain = initialize_agent(
    tools,
    llm,
    agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
    verbose=True,
    handle_parsing_errors=True
)

# Ejecutar una consulta utilizando el agente
result = agent_chain.run(
    "¿La política es agena?"
)
print(result)



> Entering new AgentExecutor chain...
Parsing LLM output produced both a final answer and a parse-able action:: Answer the following questions as best you can. You have access to the following tools:

arxiv - A wrapper around Arxiv.org Useful for when you need to answer questions about Physics, Mathematics, Computer Science, Quantitative Biology, Quantitative Finance, Statistics, Electrical Engineering, and Economics from scientific articles on arxiv.org. Input should be a search query.

Use the following format:

Question: the input question you must answer
Thought: you should always think about what to do
Action: the action to take, should be one of [arxiv]
Action Input: the input to the action
Observation: the result of the action
... (this Thought/Action/Action Input/Observation can repeat N times)
Thought: I now know the final answer
Final Answer: the final answer to the original input question

Begin!

Question: ¿La política es agena?
Thought: ¿Qué significa "es agena"? ¿Quién 

: 